# Report

Gruppe D

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.<br><br> 
Nachdem wir uns einen Überblick über die Datenquellen verschafft haben, sind wir zum Entschluss gekommen, uns die Zensus Daten von 2011 genauer anzuschauen. Die Zensus Daten 2011 enthalten unter anderem Angaben über die Erwerbstätigkeit in Deutschland sowie unterschiedlichste soziodemografische Informationen (siehe [Zensus 2011](Zensus%202011%20-%20Methoden%20und%20Verfahren.pdf)). <br><br>
Hinsichtlich der Arbeitslosigkeit gibt es diverse Vorurteile und Vermutungen. So wird häufig behauptet, dass der Grossteil der Arbeitslosen Personen mit Migrationshintergrund sind oder ein schlechtes Bildungsniveau vorweisen.<br><br>  
Die Bundeszentrale für politische Bildung veröffentlichte im Jahr 2022 hierzu einen Bericht, welcher diese zwei Vermutungen sogar bestätigt (vgl. Arbeitslosenquoten nach Geschlecht und Staatsangehoerigkeit, bpb, 2021), (vgl. Arbeitslosenquoten nach Bildung und Alter, bpb, 2021).
Diese Informationen waren für uns Anreiz genug, um diese Zusammenhänge zu untersuchen.<br><br>  
Die Fragestellung, welche wir innerhalb dieses Projekts untersuchen, lautet wie folgt:<br><br>
»Gibt es einen Zusammenhang zwischen den soziodemografischen Merkmalen und der Arbeitslosenrate?«

::: {.callout-note}
### Fragestellung
Gibt es einen Zusammenhang zwischen den soziodemografischen Merkmalen und der Arbeitslosenrate?
:::
<br>

>**Fragestellung** <br>
>Gibt es einen Zusammenhang zwischen den soziodemografischen Merkmalen und der Arbeitslosenrate?

Die Arbeitslosigkeit im Allgemeinen ist ein Indikator für die Situation auf dem Arbeitsmarkt. Die Arbeitslosenquote kann unter Angabe der "Anzahl Erwerbslosen" sowie der "Anzahl Erwerbstätigen" berechnet werden. Als soziodemografische Merkmale haben wir uns für folgende sechs entschieden und wie folgt definiert (siehe [Appendix-Predictor-Variables](#Appendix-Predictor-Variables)). Der Grund wieso wir uns für die 6 Merkmale entschieden haben war, da aufgrund der Zensus Umfrage die soziodemografischen Merkmale in verschiedene Kategorien aufgeteilt waren. Dies sind Informationen auf Gemeindeebene, wie z.B. Angaben zum Familienstand ((Anzahl Personen die ledig sind, verheiratet, usw.), Angaben zur Altersstruktur ((Anzahl Personen die unter 10 Jahre alt sind,zwischen 10-19 Jahre alt sind, usw.), Angaben nach Relegionszugehörigkeit ((Anzahl Personen die römisch-katholisch, evangelisch oder sonstiges) oder Angaben zum Bildungsniveau (Anzahl Personen ohne beruflischen Abschluss, mindestens eine Lehre, mindestens Hochschulabschluss, usw.). Aufgrund dieser Kategorien entschieden wir uns Quoten zu bilden, die unserer Meinung nach in einer Beziehung zur Arbeitslosenquote stehen können. <br><br>  

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

Zur Visualisierung der Beziehungen zwischen Response- und Predictor Variables haben wir uns für die Anwendung von Streudiagrammen bzw. Scatter Plots entschieden. Jeden Prädikator haben wir mit der Arbeitslosenquote2 (bereinigte Arbeitslosenquote) gegegnübergestellt, um erste Erkenntnisse gewinnen zu können (siehe [Appendix-EDA](#Appendix-EDA)). Jedoch ist es anhand der Scatter Plots zunächst schwierig zu erkennen, welcher Prädikator die höchste Korrelation mit der Arbeitslosigkeit aufweist, da die berechneten Variablen unterschiedliche Werte bzw. Verteilungen auf der X-Achse annehmen und kein direkter Vergleich stattfinden kann.  <br><br> 
Zu erkennen ist, dass lediglich die Variable "Christenquote" eine moderate (über 0,5 bzw. unter -0,5) Korrelation mit der Arbeitslosenquote aufweist. Zwei weitere Quoten, die noch eine leichte bis moderarte Korrelation haben sind die "Singlequote" sowie die "Migrationsquote".<br><br>   




### Import relevanter Module

In [2]:
#| echo: true 
#| code-fold: true
#import relevant modulues
import pandas as pd
import altair as alt
import numpy as np
from pandas import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows() #aus Code overview Histogramm
from scipy import stats # to compute the mode 

from sklearn.linear_model import LinearRegression #Fitting a line
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

import matplotlib.pyplot as plt  # To visualize

import joblib
import time

In [3]:
def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

### Import Datensatz

In [4]:
#| echo: true
#| code-fold: true
df_bevoelkerung = pd.read_csv(
    '../references/csv_Bevoelkerung/Zensus11_Datensatz_Bevoelkerung.csv',
    delimiter=';',
    dtype={
        'AGS_12': 'category',
        'RS_Land': 'category',
        'RS_RB_NUTS2': 'category',
        'RS_Kreis': 'category',
        'RS_VB': 'category',
        'RS_Gem': 'category',
        'Name': 'category',
        'Reg_Hier': 'category'
    },
    low_memory= False #um Warnung zu verhindern
)

### Data Structure

Um einen ersten groben Überblick zu bekommen, geben wir uns eine Info über unseren Datensatz mit der Pandas-Funktion `pd.info()` aus.

In [5]:
df_bevoelkerung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Columns: 223 entries, AGS_12 to BIL_5.8
dtypes: category(8), float64(41), int64(8), object(166)
memory usage: 21.4+ MB


Hierbei ist zu sehen, dass 166 Spalten als Datentyp `Objekt` haben und somit auf gemischte Datentypen hindeutet. Im Appendix ist die ausführlicherere Version (@sec-app_df). Hier ist ersichtlich, dass manche Spalten einige leere Zellen enthalten, so gibt es einge Spalten mit nur 2187 non-null Werten. <br><br>
Daher führen wir im nächsten Schritt Daten Korrekturen durch, um saubere Datentypen zu haben.

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

Wie in der Introduction beschrieben haben wir uns aufgrund der Literatur-Recherche wie auch nach Betrachtung des Data Sets für 6 Quoten entschieden, die die Arbeitslosenquote beeinflussen könnten. 

Wie in der explorativen Analyse zu sehen ist, hat nur die Christenquote eine Korrelation mit der Arbeitslosenquote über 0,5 bzw. unter -0,5. Zwei weitere Quoten, die noch eine moderarte bis schwache Korrelation haben sind die Singlequote sowie die Migrationsquote. Daher wurde bereits an dieser Stelle beschlossen keine weiteren Quoten für die lineare Regression mit in die Auswahl aufzunehmen. Da in den Scatterplot linere Zusammenhänge erkennbar sind, wollen wir diese mit lineran Modellen versuchen zu erklären.

Für die multiple Regression wie auch die Lasso-Regression haben wir zunächst alle Variablen als Prädikatoren ausgewählt.

Zu Beginn der jeweiligen Modellierungsprozesse haben wir jeweils das Regressionsmodell ausgewählt. Der nächste Schritt war es dann die Modelle zu validieren und an die Daten anzupassen. 
Die Validierung haben wir mit der Cross-Validation umgesetzt und dabei den Mean Squared error pro Fold angeschaut.
Bei der Lasso-Regression war die Cross-Validation implementiert.

Daraufhin haben wir mit der Funktion `reg.fit` die Modelle an die Traingsdaten angepasst und somit trainiert und den Intercept mit der y-Achse sowie den beziehungsweise die Koeffizienten berechnet.

Der abschließende Schritt war die Evaluierung mit den Test-Daten. Wir haben die Modell dann anhand verschiedener Gütemaße (R^2, MSE, RMSE, MAE) bewertet.

Durch die Peer-Review haben wir das Feedback bekommen, dass wir für die Multiple Regression die Stepwise Selection nutzen könnten sowie für die Bewertung der Güte der Lasso-Regression den AIC. 

Diese grob beschriebenen Schritte wollen wir im Folgenden detaillierter erklären.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.

Mithilfe der durchgeführten Analyse konnte festgestellt werden, dass die stärkste positive Korrelation zwischen der Arbeitslosenquote einer Gemeinde und dem Prädikator "Singlequote" besteht (r = +0.44441). Die stärkste negative Korrelation besteht zwischen der Arbeitslosenquote einer Gemeinde und dem Prädikator "Christenquote" (r = -0.66274). Die schwächste Korrelation, auf Basis unserer Daten, weist die "Akademikerquote" auf (r=-0.10260).

Das Ergebnis der drei durchgeführten Modelle lautet wie folgt: 

Lineare Regression:

Multiple Regression:

Lasso Regression: 


Die Ergebnisse lassen darauf schliessen, dass entweder die Prädikatoren nicht aussagekräftig genug sind, um die Arbeitslosigkeit erklären zu können oder das ein anderes statistische Modell gewählt werden muss.

Folgende Herausforderungen gab es bei der Durchführung des Projekts: 

1. Verknüpfung zweier Data Frames
2. Grosser Data Frame, obwohl nur wenige Daten tatsächlich relevant waren.
3. Ein Grossteil der Zellen relevanter Spalten war leer, da durch die Zensus Umfrage die Werte entweder nicht ermittelt werden konnten oder zu ungenau waren. 
4. Geringe Korrelation zwischen Arbeitslosenquote und den Variablen.

## Appendix

### Appendix df_bevoelkerung {#sec-app_df}

In [13]:
df_bevoelkerung.info(verbose = True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12544 entries, 0 to 12543
Data columns (total 223 columns):
 #    Column       Non-Null Count  Dtype   
---   ------       --------------  -----   
 0    AGS_12       12544 non-null  category
 1    RS_Land      12544 non-null  category
 2    RS_RB_NUTS2  12527 non-null  category
 3    RS_Kreis     12501 non-null  category
 4    RS_VB        12089 non-null  category
 5    RS_Gem       11339 non-null  category
 6    Name         12544 non-null  category
 7    Reg_Hier     12544 non-null  category
 8    AEWZ         12544 non-null  int64   
 9    DEM_1.1      12544 non-null  int64   
 10   DEM_1.2      12544 non-null  object  
 11   DEM_1.3      12544 non-null  object  
 12   DEM_2.1      12544 non-null  int64   
 13   DEM_2.2      12544 non-null  object  
 14   DEM_2.3      12544 non-null  object  
 15   DEM_2.4      12544 non-null  object  
 16   DEM_2.5      12544 non-null  object  
 17   DEM_2.6      12544 non-null  object  
 18   DEM_

### Appendix Predictor Variables

In [7]:
df_predictor_variables = pd.read_excel('../references/Predictor Variables Definition.xlsx', index_col= 0)

In [8]:
left_align(df_predictor_variables)

,Quote,Berechnung
Variable,,
Migrationshintergrund,Migrationsquote,(Anzahl Personen mit Migrationshintergrund / Anzahl Personen insgesamt)
Religionszugehörigkeit,Christenquote*,(Römisch-katholische Kirche + Evangelische Kirch) / Bevölkerung nach Religion gesamt
Geschlecht,Männerquote,(Anzahl Männer / Einwohner gesamt)
Bildungsniveau,Akademikerquote**,(Fach- oder Berufsakademie + FH-Abschluss + Hochschulabschluss + Promotion) / höchster beruflicher Abschluss insgesamt
Stellung im Beruf,Beamtenquote,(Anzahl Beamter / Erwerbstätige insgesamt)
Familienstand,Singlequote***,(Anzahl Lediger + Verwitwete + Geschiedene + eingetragene Lebenspartnerschaft aufgehoben + Eingetragener Lebenspartner/- in verstorben + ohne Angaben) / Familienstand gesamt


### Appendix EDA